In [1]:
import requests
from bs4 import BeautifulSoup
import json


# Загрузка текста статьи
url = "https://blog.dzencode.com/ru/illyuziya-kachestva-vash-sayt-idealen-pozdravlyaem-vy-tolko-chto-sozhgli-byudzhet/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
article_text = soup.get_text(separator='\n')

# Сохранение текста
with open('artifacts/article.txt', 'w', encoding='utf-8') as f:
    f.write(article_text)

# Метаданные
metadata = {
    "url": url,
    "language": "ru",
    "date": "unknown",  # Дата не указана, можно уточнить позже
    "topic": "Иллюзия качества в веб-дизайне",
    "project": "RAG_Pipeline_Test",
    "lang": "ru"
}
with open('artifacts/metadata.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

In [2]:
import re

# Чтение исходного текста
with open('artifacts/article.txt', 'r', encoding='utf-8') as f:
    article_text = f.read()

# Предобработка: удаление HTML-тегов и лишних символов
cleaned_text = re.sub(r'<.*?>', '', article_text)  # Удаление HTML-тегов
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Замена множества пробелов на один
cleaned_lines = cleaned_text.splitlines()  # Разбиение по любым разрывам строк
cleaned_text = '\n'.join(line.strip() for line in cleaned_lines if line.strip())  # Фильтр пустых строк

# Сохранение чистого текста
with open('artifacts/cleaned_article.txt', 'w', encoding='utf-8') as f:
    f.write(cleaned_text)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Чтение очищенного текста
with open('artifacts/cleaned_article.txt', 'r', encoding='utf-8') as f:
    cleaned_text = f.read()

# Разбиение на чанки
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len
)
chunks = text_splitter.split_text(cleaned_text)

# Подготовка данных для JSONL с ID и метаданными
data = []
for i, chunk in enumerate(chunks):
    entry = {
        "id": i,
        "text": chunk,
        "metadata": {
            "url": "https://blog.dzencode.com/ru/illyuziya-kachestva-vash-sayt-idealen-pozdravlyaem-vy-tolko-chto-sozhgli-byudzhet/",
            "language": "ru",
            "date": "unknown",
            "topic": "Иллюзия качества в веб-дизайне",
            "project": "RAG_Pipeline_Test"
        }
    }
    data.append(entry)

# Сохранение в JSONL
with open('artifacts/rag_article.jsonl', 'w', encoding='utf-8') as f:
    for entry in data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import json
import pickle
from langchain.embeddings import HuggingFaceEmbeddings

# Инициализация эмбеддингов
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Чтение чанков из JSONL
chunks = []
try:
    with open('artifacts/rag_article.jsonl', 'r', encoding='utf-8') as f:
        for line in f:
            chunks.append(json.loads(line))
except FileNotFoundError:
    print("Ошибка: Файл 'artifacts/rag_article.jsonl' не найден.")
    raise

# Генерация эмбеддингов
texts = [chunk['text'] if chunk['text'] else " " for chunk in chunks]
vectors = embeddings.embed_documents(texts)

# Подключение к Qdrant
try:
    client = QdrantClient(host='localhost', port=6334)
except Exception as e:
    print(f"Ошибка подключения к Qdrant: {e}")
    raise

# Очистка и создание коллекции
collection_name = "rag_article_collection"
if client.collection_exists(collection_name):
    client.delete_collection(collection_name)
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=len(vectors[0]),
        distance=Distance.COSINE
    )
)

# Загрузка данных в Qdrant
points = [
    PointStruct(
        id=chunk['id'],
        vector=vector,
        payload=chunk['metadata']
    )
    for chunk, vector in zip(chunks, vectors)
]
client.upsert(collection_name=collection_name, points=points)

# Сохранение эмбеддингов
with open('artifacts/embeddings.pkl', 'wb') as f:
    pickle.dump(vectors, f)

C:\Users\balnc3r\AppData\Local\Temp\ipykernel_9496\781482672.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

D:\Projects\RAG_Test\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\balnc3r\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang="ru_RU", use_fast=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    print(f"Модель {MODEL_NAME} успешно загружена")
except Exception as e:
    print(f"Ошибка загрузки модели {MODEL_NAME}: {e}")
    raise

Модель facebook/mbart-large-50-many-to-many-mmt успешно загружена


In [6]:
from qdrant_client import QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings
import json
import os
from datetime import datetime

# Инициализация эмбеддингов
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Подключение к Qdrant
client = QdrantClient(host='localhost', port=6334)
collection_name = "rag_article_collection"

# Загрузка чанков из JSONL
chunks = []
with open('artifacts/rag_article.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        chunks.append(json.loads(line))

# Словарь для доступа к тексту по id
chunk_dict = {chunk['id']: chunk['text'] for chunk in chunks}


# Поиск релевантных чанков
def search_chunks(query, top_k=3, min_score=0.4):
    query_vector = embeddings.embed_query(query)
    search_result = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=top_k,
        with_payload=True,
        with_vectors=False
    ).points
    matched_chunks = [
        chunk_dict[hit.id] for hit in search_result
        if hit.id in chunk_dict and hit.score >= min_score
    ]
    return matched_chunks


# Генерация ответа
def generate_response(query, chunks):
    if not chunks:
        return "Не удалось найти релевантный контекст."
    context = chunks[0]
    tokenized_context = tokenizer(context, truncation=True, max_length=150, return_tensors="pt")
    truncated_context = tokenizer.decode(tokenized_context["input_ids"][0], skip_special_tokens=True)
    prompt = f"""Ответь на русском языке кратко, используя контекст.

Вопрос: {query}
Контекст: {truncated_context}"""
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        num_return_sequences=1,
        do_sample=False,
        forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return response


# Проверка коллекции
collection_info = client.get_collection(collection_name=collection_name)
if collection_info.points_count == 0 or collection_info.points_count != len(chunk_dict):
    raise SystemExit


# Логирование запроса и ответа
def log_query(query, answer, chunks):
    log_entry = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "query": query,
        "answer": answer,
        "relevant_chunks": [chunk[:100] + "..." if chunk else "Пусто" for chunk in chunks]
    }
    os.makedirs('logs', exist_ok=True)
    log_file = 'logs/query_log.json'
    try:
        with open(log_file, 'r', encoding='utf-8') as f:
            logs = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        logs = []
    logs.append(log_entry)
    with open(log_file, 'w', encoding='utf-8') as f:
        json.dump(logs, f, ensure_ascii=False, indent=2)


# Обновление журнала версий
def update_version_log():
    version_entry = {
        "version": "1.0.0",
        "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "files": [
            {
                "file": "rag_article.jsonl",
                "description": "Исходный файл с чанками статей для RAG."
            },
            {
                "file": "embeddings.pkl",
                "description": "Сохранённые эмбеддинги чанков для Qdrant."
            },
            {
                "file": "rag_result.json",
                "description": "Результат последнего запроса (вопрос, ответ, релевантные чанки)."
            }
        ]
    }
    os.makedirs('artifacts', exist_ok=True)
    with open('artifacts/version_log.json', 'w', encoding='utf-8') as f:
        json.dump([version_entry], f, ensure_ascii=False, indent=2)


# Запрос
query = "Что автор имеет в виду под 'иллюзией качества'?"

# Основной поиск и генерация
relevant_chunks = search_chunks(query, top_k=3, min_score=0.4)
answer = generate_response(query, relevant_chunks)

# Сохранение результата
result = {"query": query, "answer": answer, "relevant_chunks": relevant_chunks}
os.makedirs('artifacts', exist_ok=True)
with open('artifacts/rag_result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# Логирование запроса
log_query(query, answer, relevant_chunks)

# Обновление журнала версий
update_version_log()

# Вывод
print(f"\nВопрос: {query}")
print(f"Ответ: {answer}")
print("Релевантные чанки:")
for i, chunk in enumerate(relevant_chunks, 1):
    print(f"Чанк {i}: {chunk[:100]}..." if chunk else f"Чанк {i}: Пусто")


Вопрос: Что автор имеет в виду под 'иллюзией качества'?
Ответ: Внутри — ядро: логика, смысл, результат. Иллюзия Качества — это когда вы покупате шелуху, перепутав ее с сердцевиной. Вы смотрите на сайт, как на витрину — и верите, что это бизнес-инструмент. А на самом деле — просто декор.
Релевантные чанки:
Чанк 1: рассекаем “Качество” Иллюзия Качества — это когда вы платите за лоск, а получаете пустоту Качество ц...
Чанк 2: анонимных перфекционистов Кстати, о “памятниках”. Какой самый вопиющий пример “Иллюзии качества” вы ...
Чанк 3: 1: Поверхностное качество (Витрина) Что это? Всё, что можно оценить за 5 секунд, не вникая: пиксель-...
